In [1]:
import argparse
import time
import os
import torch
from PIL import Image
from torch.autograd import Variable
from torchvision.transforms import ToTensor, ToPILImage
import torch.nn.functional as F
from model import Generator

In [2]:
UPSCALE_FACTOR = 4
TEST_IMAGE = 'lr2.jpg'
TEST_PATH = './data/test/'
OUTPUT_PATH = './test_output/'
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)
MODEL = './epochs/netG_epoch_4_47.pth'

In [3]:
model = Generator(UPSCALE_FACTOR).eval()

model.load_state_dict(torch.load(MODEL, map_location="cuda:0"))
model = model.cuda()

In [4]:
image = Image.open(TEST_PATH + TEST_IMAGE)
with torch.no_grad():
    image = Variable(ToTensor()(image)).unsqueeze(0)
    image = F.interpolate(image, size=(450, 800), mode='bicubic', align_corners=False)
    print(image.size())
image = image.cuda()

torch.Size([1, 3, 450, 800])


In [5]:
start = time.process_time()
out = model(image)
elapsed = time.process_time() - start
print('process time : ' + str(elapsed) + 's')
out = F.interpolate(out, size=(1440, 2560), mode='bicubic', align_corners=False)
out_img = ToPILImage()(out[0].data.cpu())
out_img.save(OUTPUT_PATH + TEST_IMAGE)

process time : 2.265625s
